In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.preprocessing import LabelEncoder
import warnings

# Suprime o warning de F-score/Recall/Precision se houver zero_division
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.metrics')

def avaliar_modelo_knn_classificacao(nome_arquivo_base, coluna_alvo, n_vizinhos=5):
    """
    Carrega uma base de dados, treina um modelo KNN Classifier e calcula
    Acurácia geral e métricas detalhadas (F1, Recall, Precision) por classe.

    :param nome_arquivo_base: Nome do arquivo CSV (ex: 'Base_KNNImputer.csv').
    :param coluna_alvo: Nome da coluna a ser usada como variável Y (alvo - deve ser categórica/binária).
    :param n_vizinhos: Número de vizinhos (k) para o KNN Classifier.
    :return: Dicionário com as métricas agregadas (Weighted), para a comparação final.
    """
    print(f"\n--- Avaliando Base: {nome_arquivo_base} com KNN Classifier (k={n_vizinhos}) ---")
    
    # Dicionário padrão para retornar em caso de erro
    METRICAS_PADRAO = {'Acurácia': np.nan, 'F1-Score': np.nan, 'Recall': np.nan, 'Precision': np.nan}
    
    try:
        df = pd.read_csv(nome_arquivo_base)
        
        # ⚠️ VERIFICAÇÃO CRÍTICA: Coluna alvo
        if coluna_alvo not in df.columns:
            print(f"❌ ERRO: Coluna alvo '{coluna_alvo}' não encontrada em {nome_arquivo_base}.")
            return METRICAS_PADRAO
        
        Y = df[coluna_alvo].copy()
        
        # 1. Pré-processamento do Alvo (Y): Garante que seja numérico (Label Encoding)
        le = LabelEncoder()
        try:
            # Remove NaNs da coluna alvo, se houver, para garantir o Label Encoding correto
            Y_cleaned = Y.dropna()
            
            if Y_cleaned.empty:
                print(f"❌ ERRO: Coluna alvo '{coluna_alvo}' está vazia ou contém apenas NaNs.")
                return METRICAS_PADRAO
                
            Y_encoded = le.fit_transform(Y_cleaned)
            class_labels = le.classes_ # Os rótulos originais (ex: 'Sim', 'Não', 0, 1)
            num_classes = len(class_labels)
            
            # Filtra o DataFrame X para corresponder ao Y limpo, contendo apenas colunas numéricas
            X = df.loc[Y_cleaned.index].drop(columns=[coluna_alvo]).select_dtypes(include=np.number)
            
            # Verifica se há dados suficientes
            if X.shape[0] < 2 or X.shape[1] == 0:
                print("❌ ERRO: Dados insuficientes para treinamento (amostras < 2 ou 0 features).")
                return METRICAS_PADRAO
            
            print(f"   ({num_classes} classes detectadas na coluna alvo '{coluna_alvo}': {class_labels.tolist()})")

        except Exception as e:
            print(f"❌ ERRO ao codificar a coluna alvo '{coluna_alvo}': {e}")
            return METRICAS_PADRAO


        # 2. Divisão em Treino e Teste (Usando stratify para manter a proporção das classes)
        X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y_encoded, test_size=0.3, random_state=42, stratify=Y_encoded
        )

        # 3. Treinamento do Modelo KNN
        knn = KNeighborsClassifier(n_neighbors=n_vizinhos)
        knn.fit(X_train, Y_train)

        # 4. Previsão
        Y_pred = knn.predict(X_test)
        
        print(f"✅ Previsão de classificação realizada com sucesso.")
        
        # --- CÁLCULO DAS MÉTRICAS GERAIS E AGREGADAS ---
        average_metric_final = 'weighted' # Weighted é bom para comparação agregada
        
        acuracia = accuracy_score(Y_test, Y_pred)
        f1_weighted = f1_score(Y_test, Y_pred, average=average_metric_final, zero_division=0)
        recall_weighted = recall_score(Y_test, Y_pred, average=average_metric_final, zero_division=0)
        precision_weighted = precision_score(Y_test, Y_pred, average=average_metric_final, zero_division=0)
        
        print(f"   Acurácia Geral do Modelo KNN: {acuracia:.4f}")
        print(f"   (Métricas Agregadas ({average_metric_final}): F1={f1_weighted:.4f}, Recall={recall_weighted:.4f}, Precision={precision_weighted:.4f})")
        
        # --- CÁLCULO DAS MÉTRICAS POR CLASSE (average=None) ---
        
        f1_per_class = f1_score(Y_test, Y_pred, average=None, zero_division=0)
        recall_per_class = recall_score(Y_test, Y_pred, average=None, zero_division=0)
        precision_per_class = precision_score(Y_test, Y_pred, average=None, zero_division=0)
        
        # Monta um DataFrame para exibição clara
        df_metrics = pd.DataFrame({
            'Classe': class_labels,
            'F1-Score': f1_per_class,
            'Recall': recall_per_class,
            'Precision': precision_per_class
        }).set_index('Classe')

        print("\n   📊 **MÉTRICAS DETALHADAS POR CLASSE**")
        print("   ----------------------------------------")
        print(df_metrics.round(4))
        print("   ----------------------------------------")

        # Retorna as métricas agregadas (weighted) para a tabela de comparação final
        return {'Acurácia': acuracia, 'F1-Score': f1_weighted, 'Recall': recall_weighted, 'Precision': precision_weighted}

    except FileNotFoundError:
        print(f"❌ ERRO: Arquivo '{nome_arquivo_base}' não encontrado. Certifique-se de que a imputação foi salva.")
        return METRICAS_PADRAO
    except Exception as e:
        print(f"❌ Ocorreu um erro ao processar {nome_arquivo_base}: {e}")
        return METRICAS_PADRAO


# --- EXECUÇÃO PRINCIPAL COM SEUS DADOS ---
if __name__ == "__main__":
    
    # 🎯 CONFIGURAÇÕES
    COLUNA_ALVO = 'ARTRITEC' 
    K_VIZINHOS = 9 
    
    print("--- Teste de Métrica de Modelo Preditivo (KNN Classifier) ---")
    print("--- Detalhamento por Classe da variável '{}' ---".format(COLUNA_ALVO))
    
    # 1. Avalia a base imputada pelo KNNImputer
    # Os detalhes por classe serão impressos DENTRO da função
    metricas_knn_base = avaliar_modelo_knn_classificacao(
        nome_arquivo_base="BaseKNNImputerFinal.csv", 
        coluna_alvo=COLUNA_ALVO,
        n_vizinhos=K_VIZINHOS
    )

    # 2. Avalia a base imputada pelo MissForest
    # Os detalhes por classe serão impressos DENTRO da função
    metricas_mf_base = avaliar_modelo_knn_classificacao(
        nome_arquivo_base="BaseMissForestFinal.csv", 
        coluna_alvo=COLUNA_ALVO,
        n_vizinhos=K_VIZINHOS
    )

    # --- Comparação Final (Agregada) ---
    print("\n" + "="*90)
    print("🎯 **COMPARAÇÃO DE PERFORMANCE DO MODELO KNN CLASSIFIER (Métricas Agregadas)**")
    print(f"   (Utilizando '{COLUNA_ALVO}' como Variável Alvo e k={K_VIZINHOS} vizinhos)")
    print("="*90)
    
    comparacao = pd.DataFrame({
        'Imputação': ['KNNImputer', 'MissForest'],
        'Acurácia': [metricas_knn_base['Acurácia'], metricas_mf_base['Acurácia']],
        'F1-Score (Weighted)': [metricas_knn_base['F1-Score'], metricas_mf_base['F1-Score']],
        'Recall (Weighted)': [metricas_knn_base['Recall'], metricas_mf_base['Recall']],
        'Precision (Weighted)': [metricas_knn_base['Precision'], metricas_mf_base['Precision']]
    }).set_index('Imputação')
    
    print(comparacao.round(4))
    print("\n* Para todas as métricas, quanto maior o valor (mais próximo de 1.0), melhor a performance do modelo.")
    print("* As métricas 'Weighted' consideram o desbalanceamento das classes na agregação.")


--- Teste de Métrica de Modelo Preditivo (KNN Classifier) ---
--- Detalhamento por Classe da variável 'ARTRITEC' ---

--- Avaliando Base: BaseKNNImputerFinal.csv com KNN Classifier (k=9) ---
   (2 classes detectadas na coluna alvo 'ARTRITEC': [0, 1])
✅ Previsão de classificação realizada com sucesso.
   Acurácia Geral do Modelo KNN: 0.9162
   (Métricas Agregadas (weighted): F1=0.8789, Recall=0.9162, Precision=0.8769)

   📊 **MÉTRICAS DETALHADAS POR CLASSE**
   ----------------------------------------
        F1-Score  Recall  Precision
Classe                             
0         0.0296  0.0153     0.4286
1         0.9562  0.9981     0.9176
   ----------------------------------------

--- Avaliando Base: BaseMissForestFinal.csv com KNN Classifier (k=9) ---
   (2 classes detectadas na coluna alvo 'ARTRITEC': [0, 1])
✅ Previsão de classificação realizada com sucesso.
   Acurácia Geral do Modelo KNN: 0.9159
   (Métricas Agregadas (weighted): F1=0.8783, Recall=0.9159, Precision=0.8716)

 